# 1. Settings

In [1]:
#@title 1. Install requirements
%%capture
! pip install --upgrade google-cloud-discoveryengine


import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [1]:
#@title 2. Authentication

from google.cloud import storage
from google.colab import auth as google_auth
from google.cloud import discoveryengine_v1beta as ds
from google.api_core.client_options import ClientOptions

PROJECT_ID = ""
LOCATION = ""

google_auth.authenticate_user(project_id=PROJECT_ID)

In [ ]:
#@title 3. Utility functions

def create_data_store(
    project_id: str,
    location: str,
    data_store_name: str,
    data_store_id: str
    ):

    client_options = (
        ClientOptions(api_endpoint=f"{LOCATION}-discoveryengine.googleapis.com")
        if LOCATION != "global"
        else None
    )

    client = ds.DataStoreServiceClient(client_options=client_options)

    data_store = ds.DataStore(
        display_name=data_store_name,
        industry_vertical="GENERIC",
        content_config="CONTENT_REQUIRED",
    )

    request = ds.CreateDataStoreRequest(
        parent=ds.DataStoreServiceClient.collection_path(
            project_id, location, "default_collection"
        ),
        data_store=data_store,
        data_store_id=data_store_id,
    )
    operation = client.create_data_store(request=request)

    try:
        response = operation.result(timeout=90)
    except:
        print("[ PROCESS ]long-running operation")
    else:
      print(f"[ SUCESS ] Datastore {data_store_name} created.")


def import_documents(
    project_id: str,
    location: str,
    data_store_id: str,
    gcs_uri: str,
):

    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    client = ds.DocumentServiceClient(client_options=client_options)

    parent = client.branch_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        branch="default_branch",
    )

    source_documents = [f"{gcs_uri}/*"]

    request = ds.ImportDocumentsRequest(
        parent=parent,
        gcs_source=ds.GcsSource(
            input_uris=source_documents, data_schema="content"
        ),
        # Options: `FULL`, `INCREMENTAL`
        reconciliation_mode=ds.ImportDocumentsRequest.ReconciliationMode.INCREMENTAL,
    )

    try:
      operation = client.import_documents(request=request)
      response = operation.result()
    except:
        print("[ FAILED ] Operation failed")
    else:
      print(f"[ SUCESS ] Documents imported.")


def create_engine(
    project_id: str,
    location: str,
    data_store_name: str,
    data_store_id: str
    ):

    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )
    client = ds.EngineServiceClient(client_options=client_options)

    config = ds.Engine.SearchEngineConfig(
        search_tier="SEARCH_TIER_ENTERPRISE", search_add_ons=["SEARCH_ADD_ON_LLM"]
    )

    engine = ds.Engine(
        display_name=data_store_name,
        solution_type="SOLUTION_TYPE_SEARCH",
        industry_vertical="GENERIC",
        data_store_ids=[data_store_id],
        search_engine_config=config,
    )

    request = ds.CreateEngineRequest(
        parent=ds.DataStoreServiceClient.collection_path(
            project_id, location, "default_collection"
        ),
        engine=engine,
        engine_id=engine.display_name,
    )

    try:
        operation = client.create_engine(request=request)
        response = operation.result(timeout=90)
    except:
        print("[ PROCESS ]long-running operation")
    else:
      print(f"[ SUCESS ] Vertex Search App created.")

# 2. Create Vertex Search Apps

In [ ]:
#@title 1. Vertex Search App: Mexican

DATA_STORE_NAME = "mexican-datastore"
DATA_STORE_ID = f"{DATA_STORE_NAME}-id"
GCS_URI = "gs://food-datastore/mexican"

# Create Data Store
create_data_store(
    project_id=PROJECT_ID,
    location=LOCATION,
    data_store_name=DATA_STORE_NAME,
    data_store_id=DATA_STORE_ID
    )

# Import Documents
import_documents(
    project_id=PROJECT_ID,
    location=LOCATION,
    data_store_id=DATA_STORE_ID,
    gcs_uri= (GCS_URI)
    )

# Create Vertex Search Engine App
create_engine(
    project_id=PROJECT_ID,
    location=LOCATION,
    data_store_name=DATA_STORE_NAME,
    data_store_id=DATA_STORE_ID)

[ SUCESS ] Datastore mexican-datastore created.
[ SUCESS ] Documents imported.
[ SUCESS ] Vertex Search App created.


In [ ]:
#@title 2. Vertex Search App: Peruvian

DATA_STORE_NAME = "peru-datastore"
DATA_STORE_ID = f"{DATA_STORE_NAME}-id"
GCS_URI = "gs://food-datastore/peruvian"

# Create Data Store
create_data_store(
    project_id=PROJECT_ID,
    location=LOCATION,
    data_store_name=DATA_STORE_NAME,
    data_store_id=DATA_STORE_ID
    )


# Import Documents
import_documents(
    project_id=PROJECT_ID,
    location=LOCATION,
    data_store_id=DATA_STORE_ID,
    gcs_uri= (GCS_URI)
    )

# Create Vertex Search Engine App
create_engine(
    project_id=PROJECT_ID,
    location=LOCATION,
    data_store_name=DATA_STORE_NAME,
    data_store_id=DATA_STORE_ID)

[ SUCESS ] Datastore peru-datastore created.
[ SUCESS ] Documents imported.
[ SUCESS ] Vertex Search App created.


In [ ]:
#@title 3. Vertex Search App: Argentine

DATA_STORE_NAME = "argentine-datastore"
DATA_STORE_ID = f"{DATA_STORE_NAME}-id"
GCS_URI = "gs://food-datastore/argentine"

# Create Data Store
create_data_store(
    project_id=PROJECT_ID,
    location=LOCATION,
    data_store_name=DATA_STORE_NAME,
    data_store_id=DATA_STORE_ID
    )


# Import Documents
import_documents(
    project_id=PROJECT_ID,
    location=LOCATION,
    data_store_id=DATA_STORE_ID,
    gcs_uri= (GCS_URI)
    )

# Create Vertex Search Engine App
create_engine(
    project_id=PROJECT_ID,
    location=LOCATION,
    data_store_name=DATA_STORE_NAME,
    data_store_id=DATA_STORE_ID)

[ SUCESS ] Datastore argentine-datastore created.
[ SUCESS ] Documents imported.
[ SUCESS ] Vertex Search App created.


# 3. Querying Search Apps

In [2]:
#@title 1. Init Discovery Engine client

client = ds.SearchServiceClient()

In [55]:
#@title 2. Utils

def search(
    data_store_id: str,
    query: str):

  serving_config = client.serving_config_path(
      project=PROJECT_ID,
      location=LOCATION,
      data_store=data_store_id,
      serving_config="default_config")


  content_search_spec = {
      "extractive_content_spec": {
          "max_extractive_answer_count": 3,
          "max_extractive_segment_count": 3,
          "return_extractive_segment_score": True,
          },
      }

  request = ds.SearchRequest(
      serving_config=serving_config,
      query=query,
      query_expansion_spec={ "condition": "AUTO" },
      spell_correction_spec={ "mode": "AUTO" },
      content_search_spec=content_search_spec,
      page_size=1)

  response = client.search(request)

  top_response = ""

  try:
    for r in response.results[0].document.derived_struct_data.get("extractive_segments"):
      top_response = r.get("content")
      break
  except:
    top_response = "No answer found"
  finally:
    return top_response

In [68]:
#@title 3. Query
query = ""

mexican_answer = search(data_store_id="mexican-datastore-id", query=query)
peruvian_answer = search(data_store_id="peru-datastore-id", query=query)
argentine_answer = search(data_store_id="argentine-datastore-id", query=query)

# 4. Summarize with Gemini

In [40]:
import vertexai
from vertexai.generative_models import GenerativeModel
from IPython.display import display, Markdown

vertexai.init(project=PROJECT_ID, location="us-central1")

model = GenerativeModel("gemini-1.5-flash-002")

In [69]:
prompt = f"""
You are an expert on Latin American food. Your task is to summarize and highlight
aspects of the food:

Mexican: {mexican_answer}

Peruvian: {peruvian_answer}

Argentine: {argentine_answer}

Make sure that the summary is short and concise for each country. Split the
answer in bullets. Please, return the answer in spanish. Do not invent answers
if you do not have enough information for eahc country.
"""

summary = model.generate_content(contents=[prompt])

display(Markdown(summary.text))

* **México:** La bebida de chocolate azteca, elaborada con cacao tostado, maíz y vainilla, fue transformada por los españoles al añadir azúcar, canela y vainilla, creando el chocolate caliente moderno. El tequila, bebida nacional mexicana, se obtiene del maguey, una planta que los antiguos pobladores ya usaban para bebidas alcohólicas, aunque los españoles refinaron el proceso con la destilación.

* **Perú:** La escena culinaria peruana se representa a través de una variedad de bares que ofrecen el pisco sour, cóctel nacional, en diversas presentaciones y combinaciones (con coca, maracuyá, etc.), acompañados de platos ligeros para compartir.  Las opciones incluyen bares tradicionales como El Bolivarcito y locales modernos como Huaringas y Ayahuasca.

* **Argentina:** Las empanadas, de origen hispano-portugués, son un pilar de la gastronomía argentina, un alimento práctico y sabroso con rellenos diversos (carne, jamón y queso, frutas), consumido desde hace siglos, popular entre trabajadores y en celebraciones familiares.
